In [ ]:
import os
import pyzbar.pyzbar as pyzbar
import cv2
import numpy as np
import re
from tkinter import filedialog
from tkinter import Tk


def decode(image):
    # decode the QR code from the image
    decodedObjects = pyzbar.decode(image)
    
    # print the type and data of the decoded object
    for obj in decodedObjects:
        print("Type:", obj.type)
        print("Data: ", obj.data)
        
    return decodedObjects

def display(image, decodedObjects):
    # loop over the decoded objects
    for obj in decodedObjects:
        # get the points of the QR code
        points = np.array(obj.polygon)

        # draw a convex hull around the points
        hull = cv2.convexHull(points)
        
        # draw the convex hull on the image
        cv2.drawContours(image, [hull], -1, (0, 255, 0), 2)

def does_file_exist_in_dir(directory):
    # check if any files exist in the directory
    return len(os.listdir(directory)) > 0

# prompt the user to select the directory containing the images
root = Tk()
root.withdraw()
directory = filedialog.askdirectory()

# check if the directory is empty
if not does_file_exist_in_dir(directory):
    print("The selected directory is empty.")
    exit()

# read through all the files in the directory
for file in os.listdir(directory):
    # skip hidden files
    if file[0] == ".":
        continue
        
    # get the full path of the file
    file_path = os.path.join(directory, file)
    
    # read the image
    image = cv2.imread(file_path)
        
    # decode the QR code from the image
    decodedObjects = decode(image)
    
    # display the location of the QR code in the image
    display(image, decodedObjects)
    
   # rename the image according to the content of the QR code
    if len(decodedObjects) > 0:
        # get the data from the first decoded object
        data = decodedObjects[0].data.decode("utf-8")
        
        # replace all non-alphanumeric characters in the data with underscores
        data = re.sub(r'[^\w]', '_', data)

        # create a new file name based on the data from the QR code
        new_file_name = data + '.jpg'
        
        # if there are multiple images with the same QR code, add a unique character to the end of each file name
        count = 1
        while os.path.exists(os.path.join(directory, new_file_name)):
            new_file_name = data + str(count) + '.jpg'
            count += 1
        
        # rename the file
        os.rename(file_path, os.path.join(directory, new_file_name))
        
    else:
        # if no QR code was found, print a message
        print("No QR code found in", file)